In [1]:
pip install tensorflow matplotlib pillow

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls /content/drive/MyDrive/dataset-civiltà-contadina

CivContadina_serie1  CivContadina_serie2  CivContadina_serie3


In [ ]:
# Esempio di addestramento di una GAN
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
import numpy as np

# Modifica dell'architettura del generatore per immagini 32x32
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128 * 8 * 8, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((8, 8, 128)))
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Conv2D(3, (3, 3), activation='tanh', padding='same'))  # Output di dimensione 32x32x3 per CIFAR-10
    return model


from keras import layers, models

# Funzione per costruire il discriminatore
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', input_shape=(32, 32, 3)))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv2D(128, (3, 3), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Definisci la GAN combinando generatore e discriminatore
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

from keras.optimizers import Adam

# Compila il modello GAN
def compile_gan(gan, discriminator, latent_dim):
    discriminator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5), metrics=['accuracy'])
    gan.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

# Funzione per addestrare la GAN
def train_gan(generator, discriminator, gan, dataset, latent_dim, n_epochs=100, batch_size=128):
    compile_gan(gan, discriminator, latent_dim)

    bat_per_epo = int(dataset.shape[0] / batch_size)
    half_batch = int(batch_size / 2)

    for i in range(n_epochs):
        for j in range(bat_per_epo):
            # Seleziona un batch casuale di immagini reali
            idx = np.random.randint(0, dataset.shape[0], half_batch)
            real_images = dataset[idx]

            # Genera un batch casuale di immagini fake
            noise = np.random.normal(0, 1, (half_batch, latent_dim))
            fake_images = generator.predict(noise)

            # Addestra il discriminatore
            x = np.concatenate((real_images, fake_images))
            y = np.concatenate((np.ones((half_batch, 1)), np.zeros((half_batch, 1))))
            d_loss = discriminator.train_on_batch(x, y)

            # Genera un batch di input per la GAN
            x_gan = np.random.normal(0, 1, (batch_size, latent_dim))
            y_gan = np.ones((batch_size, 1))

            # Addestra la GAN (il discriminatore viene congelato durante l'addestramento)
            g_loss = gan.train_on_batch(x_gan, y_gan)

            # Stampa il progresso dell'addestramento
            print(f'Epoch {i+1}, Batch {j+1}/{bat_per_epo}, D_loss={d_loss}, G_loss={g_loss}')


# Esempio di utilizzo
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# Carica il dataset, ad esempio con cifar10
from tensorflow.keras.datasets import cifar10
(train_X, _), (_, _) = cifar10.load_data()

# Normalizza le immagini
train_X = (train_X.astype('float32') - 127.5) / 127.5

# Addestra la GAN
train_gan(generator, discriminator, gan, train_X, latent_dim)



170498071/170498071 [==============================] - 4s 0us/step


Streaming output truncated to the last 5000 lines.
Epoch 2, Batch 61/390, D_loss=[2.367746114730835, 0.1640625], G_loss=0.01955525204539299
2/2 [==============================] - 0s 145ms/step
Epoch 2, Batch 62/390, D_loss=[2.3665719032287598, 0.171875], G_loss=0.019787762314081192
2/2 [==============================] - 0s 144ms/step
Epoch 2, Batch 63/390, D_loss=[2.3561835289001465, 0.1796875], G_loss=0.01943342015147209
2/2 [==============================] - 0s 155ms/step
Epoch 2, Batch 64/390, D_loss=[2.3396787643432617, 0.1875], G_loss=0.019565407186746597
2/2 [==============================] - 0s 153ms/step
Epoch 2, Batch 65/390, D_loss=[2.382701873779297, 0.1953125], G_loss=0.01964040845632553
2/2 [==============================] - 1s 249ms/step
Epoch 2, Batch 66/390, D_loss=[2.3593263626098633, 0.1953125], G_loss=0.020025432109832764
2/2 [==============================] - 0s 153ms/step
Epoch 2, Batch 67/390, D_loss=[2.330406665802002, 0.2265625], G_loss=0.018967404961586
2/2 [==

In [ ]:
#DOPO L'ADDESTRAMENTO

# Carica l'immagine di input
input_image_path = '/content/drive/MyDrive/dataset-civiltà-contadina/CivContadina_serie2/197.jpg'
img = Image.open(input_image_path)
img = img.convert('RGB')
img = img.resize((128, 128))  # Assicurati di ridimensionare all'input del generatore

# Prepara l'immagine per l'input del generatore
input_image = np.array(img)
input_image = input_image / 255.0  # Normalizzazione

# Genera un'immagine simile utilizzando il generatore
latent_dim = 100  # Dimensione del vettore di rumore
noise = np.random.normal(0, 1, (1, latent_dim))
generated_image = generator.predict(noise)

# Visualizza l'immagine generata
plt.subplot(1, 2, 1)
plt.imshow(input_image)
plt.title('Input Image')

plt.subplot(1, 2, 2)
plt.imshow(generated_image[0])
plt.title('Generated Image')

plt.tight_layout()
plt.show()